In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np
import math
import matplotlib.pyplot as plt
import random

In [ ]:
from titanic_analysis.functions import extract_title, fillna_age

In [ ]:
titanic_data = pd.read_csv('titanic.csv')

In [ ]:
titanic_data.drop(['PassengerId', 'Ticket'], axis=1, inplace=True)

In [ ]:
titanic_data.head()

In [ ]:
print('Zero values in train name = ' + str(titanic_data['Name'].isnull().sum()))
print('Zero values in train sex = ' + str(titanic_data['Sex'].isnull().sum()))
print('Zero values in train cabin = ' + str(titanic_data['Cabin'].isnull().sum()))
print('Zero values in train embarked = ' + str(titanic_data['Embarked'].isnull().sum()))

In [ ]:
titanic_data.describe()

# Extract title out of name

In [ ]:
titanic_data = extract_title(titanic_data)

# Fill in the small amount of missing values for fare

In [ ]:
titanic_data['Fare'].fillna(titanic_data['Fare'].mean(), inplace=True)

# Filling in missing values for age

In [ ]:
titanic_data = fillna_age(titanic_data)

# Extract cabin class out of cabin

In [ ]:
for i in range(len(titanic_data)):
    if type(titanic_data['Cabin'][i]) == str:
        trimmed = titanic_data['Cabin'][i][:1]
        titanic_data.loc[i,'Cabin'] = trimmed

# Filling in missing values for cabin

In [ ]:
cabin = titanic_data[['Cabin','SibSp','Parch','Age','Fare','Pclass']].dropna()
cabin['Fare'] = cabin['Fare'].astype(int)
grouped_cabin = cabin.groupby(['Cabin'])['Pclass'].mean()
print(grouped_cabin)

In [ ]:
# Can be improved
classone = ['A', 'B', 'C', 'D', 'E', 'T']
for i in range(len(titanic_data)):
    if not type(titanic_data['Cabin'][i]) == str:
        if titanic_data.loc[i,'Pclass'] == 1:
            titanic_data.loc[i,'Cabin'] = random.choice(classone)
        elif titanic_data.loc[i,'Pclass'] == 2:
            titanic_data.loc[i,'Cabin'] = 'F'
        elif titanic_data.loc[i,'Pclass'] == 3:
            titanic_data.loc[i,'Cabin'] = 'G'

# Visualizations

In [ ]:
titanic_data.columns

In [ ]:
import seaborn as sns

sns.barplot(y=titanic_data['Survived'], x=titanic_data['Pclass'])

In [ ]:
sns.barplot(y=titanic_data['Survived'], x=titanic_data['Name'])

In [ ]:
sns.barplot(y=titanic_data['Survived'], x=titanic_data['Sex'])

In [ ]:
sns.barplot(x=titanic_data['Survived'], y=titanic_data['Age'])

In [ ]:
sns.barplot(x=titanic_data['Survived'], y=titanic_data['Fare'])

In [ ]:
sns.barplot(x=titanic_data['Survived'], y=titanic_data['Cabin'])

In [ ]:
sns.barplot(x=titanic_data['Survived'], y=titanic_data['Embarked'])

In [ ]:
sns.barplot(y=titanic_data['Survived'], x=titanic_data['SibSp'])

In [ ]:
sns.barplot(y=titanic_data['Survived'], x=titanic_data['Parch'])

# SibSp + Parch into familysize

In [ ]:
titanic_data['FamilySize'] = titanic_data['SibSp'] + titanic_data['Parch'] + 1
titanic_data.drop(['SibSp'], axis=1, inplace=True)
titanic_data.drop(['Parch'], axis=1, inplace=True)

In [ ]:
sns.barplot(y=titanic_data['Survived'], x=titanic_data['FamilySize'])

# Combine FamilySize 8 + 11 because both have 0 change of surviving

In [ ]:
for i in range(len(titanic_data)):
    if titanic_data.loc[i,'FamilySize'] == 11:
        titanic_data.loc[i,'FamilySize'] = 8